In [0]:
import tensorflow as tf
from csv import reader


In [0]:
# Read datasets from CSV input file
def Read_file(file_name):
    dataset = list()
    with open(file_name, 'r',encoding='utf-8') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [0]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

In [0]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])


In [21]:
from google.colab import files
uploaded = files.upload()

Saving GTD_Data.csv to GTD_Data.csv
Saving GTD_Target.csv to GTD_Target.csv


In [22]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

User uploaded file "GTD_Data.csv" with length 1690348 bytes
User uploaded file "GTD_Target.csv" with length 827946 bytes


In [0]:
# Input dataset
Data = Read_file('GTD_Data.csv')
Target = Read_file('GTD_Target.csv')

In [0]:
# Convert string columns to float in input dataset
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
# change string column values to float
for i in range(len(Data[0])):
    str_column_to_float(Data, i)

#normalize dataset to get better result
minmax = dataset_minmax(Data)
normalize_dataset(Data, minmax)

In [0]:
import numpy as np
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

Data, Target = next_batch(len(Data),Data,Target)
traindata = Data[0:140000]
traintarget = Target[0:140000]
testdata = Data[140000:]
testtarget = Target[140000:]

In [0]:
n_nodes_hl1 = 1000
n_nodes_hl2 = 2000
n_nodes_hl3 = 500

n_classes = 3453
batch_size = 2048

In [0]:
x = tf.placeholder('float',[None,4])
y = tf.placeholder('int64')

In [0]:
def neural_network_model(data):

    # (input_data * weight)  + bias
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([4, n_nodes_hl1])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'biases': tf.Variable(tf.random_normal([n_classes]))}

    #input_data * weight +b
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights'])+  output_layer['biases']

    return output




In [30]:
import numpy as np


def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.losses.softmax_cross_entropy(tf.one_hot(y, 3453), logits=prediction))
    # Adam optimizer does has learning rate parameter
    optimizer = tf.train.AdamOptimizer().minimize(cost)


    #cycles for feed forward and backprop error
    nm_epochs = 50

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(nm_epochs):
            epoch_loss = 0
            for _ in range(int(len(Data)/batch_size)):
                epoch_x,epoch_y = next_batch(batch_size,traindata,traintarget)
                _ ,c = sess.run([optimizer,cost], feed_dict= {x :epoch_x, y : epoch_y})
                epoch_loss += c
            print("Epoch", (epoch + 1),' completed out of ', nm_epochs , "Loss:" , epoch_loss)
        correct_prediction = tf.equal(tf.argmax(prediction, 1), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        for i in range(0,10):
          next_d,next_t = next_batch(1000,testdata,testtarget)
          print(sess.run(accuracy, feed_dict={x:next_d, y: next_t}))
        print(sess.run(accuracy, feed_dict={x:traindata[0:1000], y: traintarget[0:1000]}))
        print("end of execution")

train_neural_network(x)

Epoch 1  completed out of  50 Loss: 908983.7607421875
Epoch 2  completed out of  50 Loss: 159670.3046875
Epoch 3  completed out of  50 Loss: 77794.18927001953
Epoch 4  completed out of  50 Loss: 43914.17468261719
Epoch 5  completed out of  50 Loss: 15820.316467285156
Epoch 6  completed out of  50 Loss: 1986.5895929336548
Epoch 7  completed out of  50 Loss: 1323.7825031280518
Epoch 8  completed out of  50 Loss: 1125.1194019317627
Epoch 9  completed out of  50 Loss: 1016.5598106384277
Epoch 10  completed out of  50 Loss: 903.258975982666
Epoch 11  completed out of  50 Loss: 821.9766883850098
Epoch 12  completed out of  50 Loss: 763.6552038192749
Epoch 13  completed out of  50 Loss: 705.4059414863586
Epoch 14  completed out of  50 Loss: 683.5343894958496
Epoch 15  completed out of  50 Loss: 663.847728729248
Epoch 16  completed out of  50 Loss: 643.1936058998108
Epoch 17  completed out of  50 Loss: 622.8568043708801
Epoch 18  completed out of  50 Loss: 603.3919048309326
Epoch 19  completed